In [13]:
#lib para facilitar o uso de dataframes
import pandas as pd
#lib para facilitar alguns cálculos
import numpy as np
#Lib para auxiliar o processo de count
from collections import Counter

In [14]:
#lendo o csv com o dataset pronto
df_voting = pd.read_csv('data/cleaned_voting.csv')

In [15]:
#Confirmando o formato dos dados
df_voting.head()

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,partido_democrat,partido_republican
0,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,n,n,y
1,n,n,y,n,n,y,y,y,y,n,y,n,n,n,y,y,y,n
2,y,y,n,n,n,y,y,n,n,n,y,y,n,y,n,y,y,n
3,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n,y,n
4,n,n,n,y,y,n,n,n,n,n,n,y,n,y,n,n,n,y


# Regras de associacao

Regras de associacao sao tecnicas de mineiracao de dados usadas para descobrir associacoes interessantes entre atributos de um determinado banco de dados.
A definicao classica de regras de associacao foi apresentada em Agrawal, Imielinski, & Swami (1993) and Han & Kamber, (2006)), e e definida como o seguinte:

Seja T {t1,t2...tn} um conjunto de transacoes e I {i1,i2...in} um conjunto de itens, definimos D como o dado relevante para a tarefa sendo um conjunto de transacoes onde cada T e um conjunto de itens de maneira que T seja um subconjunto de I.

Uma regra de associacao tem a forma X -> Y, onde X e Y sao conjuntos disjuntos

Em nosso dataset as transacoes T representao o conjunto de votos de cada representante, e os Itens representao a reposta ao voto para cada projeto.

O algoritmo aprior é usado para encontrar conjuntos frequentes de items em databases como o nosso.
Ele funciona com a afirmativa que:
> Um sub conjunto de um conjunto frequente provavelmente e trambem um conjunto frequente

Por exemplo:
   se {i1,i2} é um conjunto frequente entao provavelmente {i1} e {i2} tambem devem ser conjuntos de items frequentes
   
- O algoritmo faz varias iteracoes em busca de itens frequentes e usa esses itens frequentes para gerar as regras de associacao

Abaixo vemos a implementacao do algoritmo passo a passo

---

Antes de comecarmos com o algoritmo precisamos realizar a definicao das metricas

In [160]:
'''
Variavel global para definicao de limite superior para suporte
'''
SUP_THRESHOLD = 0.50

In [161]:
def get_suporte(qtd_X, qtd_total):
    '''
    suporte = Numero de transacoes que um item aparece
              ---------------------------------------
              Numero total de transacoes
    '''
    return (qtd_X*1.0)/qtd_total

def get_confianca():
    pass

## Passo 1

No primeiro passo vamos crair uma tabela com todos os counts individuais de cada votacao
positiva para cada item

In [167]:
def itens_distintos(df):
    '''
    1. Conta a quantidade de y e n em todas as colunas, pegando apenas y no final
    e Transpondo para melhorar o entendimento da tabela
    2. Adiciona uma coluna com o suporte calculado
    3. Passa o threshold de suporte retornando apenas os itens acima do suporte
    '''
    #1.
    countings = pd.DataFrame(df.apply(pd.Series.value_counts).T['y'])
    #2.
    countings['suporte'] = countings.apply(lambda k: get_suporte(k['y'], df_voting.shape[0]),axis=1)
    #3.
    countings = countings.loc[countings['suporte'] >= SUP_THRESHOLD]
    return countings

In [60]:
candidatos = itens_distintos(df_voting)

In [63]:
candidatos

,y,suporte
p_2,243,0.558621
p_3,260,0.597701
p_6,274,0.629885
p_7,247,0.567816
p_8,246,0.565517
p_9,226,0.519540
p_10,219,0.503448
p_13,219,0.503448
p_14,255,0.586207
p_16,373,0.857471
